# AML Template - sample run on remote compute

## Init & Login

In [18]:
## Check core SDK version number
import azureml.core
import mlflow
import os

from azureml.core import (Datastore, Dataset, Environment, Experiment, ScriptRunConfig,
                          Workspace)
from azureml.core.authentication import InteractiveLoginAuthentication
from IPython.display import display

print("[INFO] SDK version:", azureml.core.VERSION)

ws = Workspace.from_config()
print("[SUCCESS] LOGGED IN: ",ws.name, ws.resource_group, ws.location, ws.subscription_id, sep=' @ ')


aml_compute = "julia-cluster" # selecting an AML Compute cluster where the actual training happens
aml_experiment = "mlflow-azureml" # name the Experiment for tracking purposes
project_folder = "code/train/julia" # where is the code to run on disk
experiment = Experiment(ws, aml_experiment) # init experiment by name

[INFO] SDK version: 1.20.0
[SUCCESS] LOGGED IN:  @ mlops-demo @ mlops-rg @ westeurope @ 6ee947fa-0d77-4915-bf68-4a83a8bec2a4


## Training on AML Compute

To train on AML Remote compute taget you need specify three config steps:

1. define Compute Taget
1. define Environment
1. define Script Run

And lastly run the experiment!

In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_target = ComputeTarget(workspace=ws, name=aml_compute)
# Use the 'status' property to get a detailed status for the current cluster. 
cts = compute_target.status.serialize()
print(f'Found existing compute target: {aml_compute}\n({"cluster is running" if (int(cts["currentNodeCount"])>0) else "cluster is idle"}) currentNodeCount: {cts["currentNodeCount"]}, vmPriority: {cts["vmPriority"]}, vmSize: {cts["vmSize"]}')

Found existing compute target: julia-cluster
(cluster is idle) currentNodeCount: 0, vmPriority: Dedicated, vmSize: STANDARD_DS2_V2


In [ ]:
from azureml.core import Environment

julia_env = Environment("julia")
julia_env.docker.enabled = True

In [33]:
# FROM mcr.microsoft.com/azureml/base
# FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04

# Specify Docker steps as a string. 
dockerfile = r"""
FROM mcr.microsoft.com/azureml/base
RUN pip install jill -U
RUN jill install --confirm
RUN echo "Hello from Julia container!"
"""

# Set the base image to None, because the image is defined by Dockerfile.
julia_env.docker.base_image = None
julia_env.docker.base_dockerfile = dockerfile

# # Alternatively, load the string from a file.
# julia_env.docker.base_image = None
# julia_env.docker.base_dockerfile = "./Dockerfile"

In [39]:
from azureml.core import ScriptRunConfig

# wtds = Dataset.get_by_name(ws, name=aml_dset)

src = ScriptRunConfig(source_directory=project_folder, 
                      command='julia train.jl',
#                       script='train.jl', 
#                       arguments =['--arg1', 1],
                      compute_target=compute_target.name,
                      environment=julia_env
                     )

# src.run_config.framework = 'python'
# src.run_config.environment = julia_env
# src.run_config.target = compute_target.name

In [40]:
run = experiment.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
mlflow-azureml,mlflow-azureml_1617206321_289c65c0,azureml.scriptrun,Queued,Link to Azure Machine Learning studio,Link to Documentation
